# Day 24: Never Tell Me The Odds

## Part 1

It seems like something is going wrong with the snow-making process. Instead of forming snow, the water that's been absorbed into the air seems to be forming hail!

Maybe there's something you can do to break up the hailstones?

Due to strong, probably-magical winds, the hailstones are all flying through the air in perfectly linear trajectories. You make a note of each hailstone's position and velocity (your puzzle input). For example:

    19, 13, 30 @ -2,  1, -2
    18, 19, 22 @ -1, -1, -2
    20, 25, 34 @ -2, -2, -4
    12, 31, 28 @ -1, -2, -1
    20, 19, 15 @  1, -5, -3

Each line of text corresponds to the position and velocity of a single hailstone. The positions indicate where the hailstones are right now (at time 0). The velocities are constant and indicate exactly how far each hailstone will move in one nanosecond.

Each line of text uses the format px py pz @ vx vy vz. For instance, the hailstone specified by 20, 19, 15 @ 1, -5, -3 has initial X position 20, Y position 19, Z position 15, X velocity 1, Y velocity -5, and Z velocity -3. After one nanosecond, the hailstone would be at 21, 14, 12.

Perhaps you won't have to do anything. How likely are the hailstones to collide with each other and smash into tiny ice crystals?

To estimate this, consider only the X and Y axes; ignore the Z axis. Looking forward in time, how many of the hailstones' paths will intersect within a test area? (The hailstones themselves don't have to collide, just test for intersections between the paths they will trace.)

In this example, look for intersections that happen with an X and Y position each at least 7 and at most 27; in your actual data, you'll need to check a much larger test area. Comparing all pairs of hailstones' future paths produces the following results:

    Hailstone A: 19, 13, 30 @ -2, 1, -2
    Hailstone B: 18, 19, 22 @ -1, -1, -2
    Hailstones' paths will cross inside the test area (at x=14.333, y=15.333).

    Hailstone A: 19, 13, 30 @ -2, 1, -2
    Hailstone B: 20, 25, 34 @ -2, -2, -4
    Hailstones' paths will cross inside the test area (at x=11.667, y=16.667).

    Hailstone A: 19, 13, 30 @ -2, 1, -2
    Hailstone B: 12, 31, 28 @ -1, -2, -1
    Hailstones' paths will cross outside the test area (at x=6.2, y=19.4).

    Hailstone A: 19, 13, 30 @ -2, 1, -2
    Hailstone B: 20, 19, 15 @ 1, -5, -3
    Hailstones' paths crossed in the past for hailstone A.

    Hailstone A: 18, 19, 22 @ -1, -1, -2
    Hailstone B: 20, 25, 34 @ -2, -2, -4
    Hailstones' paths are parallel; they never intersect.

    Hailstone A: 18, 19, 22 @ -1, -1, -2
    Hailstone B: 12, 31, 28 @ -1, -2, -1
    Hailstones' paths will cross outside the test area (at x=-6, y=-5).

    Hailstone A: 18, 19, 22 @ -1, -1, -2
    Hailstone B: 20, 19, 15 @ 1, -5, -3
    Hailstones' paths crossed in the past for both hailstones.

    Hailstone A: 20, 25, 34 @ -2, -2, -4
    Hailstone B: 12, 31, 28 @ -1, -2, -1
    Hailstones' paths will cross outside the test area (at x=-2, y=3).

    Hailstone A: 20, 25, 34 @ -2, -2, -4
    Hailstone B: 20, 19, 15 @ 1, -5, -3
    Hailstones' paths crossed in the past for hailstone B.

    Hailstone A: 12, 31, 28 @ -1, -2, -1
    Hailstone B: 20, 19, 15 @ 1, -5, -3
    Hailstones' paths crossed in the past for both hailstones.

So, in this example, 2 hailstones' future paths cross inside the boundaries of the test area.

However, you'll need to search a much larger test area if you want to see if any hailstones might collide. Look for intersections that happen with an X and Y position each at least 200000000000000 and at most 400000000000000. Disregard the Z axis entirely.

Considering only the X and Y axes, check all pairs of hailstones' future paths for intersections. How many of these intersections occur within the test area?

In [7]:
def coordinates(line):

    # Get coordinates
    coord = line.strip().split("@")

    # Split coordinates into position and velocity, convert to integers
    x, y, z = map(int, map(str.strip, coord[0].split(',')))
    vx, vy, vz = map(int, map(str.strip, coord[1].split(',')))

    return x,y,z,vx,vy,vz

def intersec(file_path, dim, debug):

    # Stat counter
    count = 0

    # Read all lines into a list
    with open(file_path, 'r') as outer_file:
        all_lines = outer_file.readlines()

    # Iterate through each line
    for i, outer_line in enumerate(all_lines):

        # Get coordinates
        x_1,y_1,z_1,vx_1,vy_1,vz_1 = coordinates(outer_line)
    
        # Iterate through lines after the current line
        for inner_line in all_lines[i + 1:]:

            # Check if inner_line is empty
            if not inner_line.strip():
                break
            
            # Get coordinates
            x_2,y_2,z_2,vx_2,vy_2,vz_2 = coordinates(inner_line)

            # Calculate times of interesection
            denominator = -vy_2 * vx_1 + vx_2 * vy_1

            numerator_1 = (vy_2 * (x_1 - x_2) + vx_2 * (y_2 - y_1))
            numerator_2 = (vy_1 * (x_1 - x_2) + vx_1 * (y_2 - y_1))

            # Check if the denominator is not equal to zero
            if denominator != 0:
                t_1 =  numerator_1 / denominator
                t_2 =  numerator_2 / denominator

                # Compute intersection
                x_i = x_1 + vx_1*t_1
                y_i = y_1 + vy_1*t_1

                if dim[0] < x_i < dim[1] and  dim[0] < y_i < dim[1] and t_1 > 0 and t_2 > 0:

                    if debug:
                        print(f"Hailstone A: {x_1}, {y_1}, {z_1} @ {vx_1}, {vy_1}, {vz_1}")
                        print(f"Hailstone B: {x_2}, {y_2}, {z_2} @ {vx_2}, {vy_2}, {vz_2}")
                        print(f"Hailstones' paths crossed inside the area. at (x={x_i}, y={y_i})")
                    
                    count += 1

                elif dim[0] < x_i < dim[1] and  dim[0] < y_i < dim[1] and t_1 < 0: 
                    if debug:
                        print(f"Hailstone A: {x_1}, {y_1}, {z_1} @ {vx_1}, {vy_1}, {vz_1}")
                        print(f"Hailstone B: {x_2}, {y_2}, {z_2} @ {vx_2}, {vy_2}, {vz_2}")
                        print("Hailstones' paths crossed in the past for hailstone A.")
                    pass     

                elif dim[0] < x_i < dim[1] and  dim[0] < y_i < dim[1] and t_2 < 0: 
                    if debug:
                        print(f"Hailstone A: {x_1}, {y_1}, {z_1} @ {vx_1}, {vy_1}, {vz_1}")
                        print(f"Hailstone B: {x_2}, {y_2}, {z_2} @ {vx_2}, {vy_2}, {vz_2}")
                        print("Hailstones' paths crossed in the past for hailstone B.")
                    pass     

                else:
                    if debug:
                        print(f"Hailstone A: {x_1}, {y_1}, {z_1} @ {vx_1}, {vy_1}, {vz_1}")
                        print(f"Hailstone B: {x_2}, {y_2}, {z_2} @ {vx_2}, {vy_2}, {vz_2}")
                        print(f"Hailstones' paths will cross outside the test area. at (x={x_i}, y={y_i})")
                    pass  
            else:
                if debug:
                    print(f"Hailstone A: {x_1}, {y_1}, {z_1} @ {vx_1}, {vy_1}, {vz_1}")
                    print(f"Hailstone B: {x_2}, {y_2}, {z_2} @ {vx_2}, {vy_2}, {vz_2}")
                    print(f"Hailstones' paths are parallel; they never intersect.")   
                pass
    return count

In [8]:
# Open the file in read mode
file_path = 'input_24.txt'

intersec(file_path, [200000000000000,400000000000000], debug=False)

17867

## Part 2

Upon further analysis, it doesn't seem like any hailstones will naturally collide. It's up to you to fix that!

You find a rock on the ground nearby. While it seems extremely unlikely, if you throw it just right, you should be able to hit every hailstone in a single throw!

You can use the probably-magical winds to reach any integer position you like and to propel the rock at any integer velocity. Now including the Z axis in your calculations, if you throw the rock at time 0, where do you need to be so that the rock perfectly collides with every hailstone? Due to probably-magical inertia, the rock won't slow down or change direction when it collides with a hailstone.

In the example above, you can achieve this by moving to position 24, 13, 10 and throwing the rock at velocity -3, 1, 2. If you do this, you will hit every hailstone as follows:

    Hailstone: 19, 13, 30 @ -2, 1, -2
    Collision time: 5
    Collision position: 9, 18, 20

    Hailstone: 18, 19, 22 @ -1, -1, -2
    Collision time: 3
    Collision position: 15, 16, 16

    Hailstone: 20, 25, 34 @ -2, -2, -4
    Collision time: 4
    Collision position: 12, 17, 18

    Hailstone: 12, 31, 28 @ -1, -2, -1
    Collision time: 6
    Collision position: 6, 19, 22

    Hailstone: 20, 19, 15 @ 1, -5, -3
    Collision time: 1
    Collision position: 21, 14, 12

Above, each hailstone is identified by its initial position and its velocity. Then, the time and position of that hailstone's collision with your rock are given.

After 1 nanosecond, the rock has exactly the same position as one of the hailstones, obliterating it into ice dust! Another hailstone is smashed to bits two nanoseconds after that. After a total of 6 nanoseconds, all of the hailstones have been destroyed.

So, at time 0, the rock needs to be at X position 24, Y position 13, and Z position 10. Adding these three coordinates together produces 47. (Don't add any coordinates from the rock's velocity.)

Determine the exact position and velocity the rock needs to have at time 0 so that it perfectly collides with every hailstone. What do you get if you add up the X, Y, and Z coordinates of that initial position?

In [33]:
def coordinates(line):

    # Get coordinates
    coord = line.strip().split("@")

    # Split coordinates into position and velocity, convert to integers   
    pos = list(map(int, coord[0].split(',')))
    vel = list(map(int, coord[1].split(',')))

    return pos,vel

In [59]:
# Solution from user hcs64
import z3

# Open the file in read mode
file = open("input_24.txt")

# Initialize lists
pos = []
vel = []

for n, line in enumerate(file):
    if n == 3: # 9 equations, 9 unkowns
        break
    else:
        # Extract coordinates
        pos0, vel0 = coordinates(line)
        pos.append(pos0)
        vel.append(vel0)
        
# Define solver variables
x = z3.Real('x')
y = z3.Real('y')
z = z3.Real('z')
vx = z3.Real('vx')
vy = z3.Real('vy')
vz = z3.Real('vz')
# Solver
solver = z3.Solver()

# Loop through all (six) hailstones and build system of eq
for i in range(len(pos)):
    x_i, y_i, z_i = pos[i]
    vx_i, vy_i, vz_i = vel[i]
    t_i = z3.Real(f"t_{i}")
    solver.add(x_i + vx_i * t_i == x + vx * t_i)
    solver.add(y_i + vy_i * t_i == y + vy * t_i)
    solver.add(z_i + vz_i * t_i == z + vz * t_i)

# Check satisfiability
print(solver.check())
# Create model
model = solver.model()

# Print the values of x, y, and z from the model
print("Position of x:", model[x])
print("Position of y:", model[y])
print("Position of z:", model[z])

# Add the values of model[x] and model[y]
result = int(model[x].as_long()) + int(model[y].as_long()) + int(model[z].as_long())

# Print the sum of x and y positions
print("Sum of x, y and z positions:", result)

sat
Position of x: 159153037374407
Position of y: 228139153674672
Position of z: 170451316297300
Sum of x, y and z positions: 557743507346379
